In [35]:
from datetime import datetime, timedelta
from utils import get_prices
import pandas as pd


EPSILON = 1e-10

# if __name__ == "__main__":

start_time = '1514764800'
end_time = '1523404800'

# get_prices('btc', 'usdt',
#            datetime(2018, 1, 1),
#            duration=timedelta(days=2),
#            proxy_port='59742',
#            verbose=2)

df_ = pd.read_csv('data/BTCUSDT[' + start_time + '000_' + end_time + '000].csv')
df_.drop(['Unnamed: 0'], inplace=True, axis=1)
df_['Time'] = pd.to_datetime(df_['Time'])
df_.set_index('Time', inplace=True, drop=True)

times_index = pd.date_range(start=pd.to_datetime(start_time, unit='s'),
                            end=pd.to_datetime(end_time, unit='s'),
                            periods=(int(end_time) - int(start_time) + 1) / 60 + 1)

df = pd.DataFrame(None, columns=df_.columns, index=times_index[:-1])

selected_indices = list(set(df_.index).intersection(set(times_index)))
df.loc[selected_indices, df_.columns] = df_.loc[selected_indices].copy()

In [36]:
window = int(timedelta(hours=1) / timedelta(minutes=1))
df['Feature 1'] = df['Taker Buy Volume'] / (df['Volume'] + EPSILON) * 100
df['Feature 2'] = df['Volume'].pct_change()
df['Feature 3'] = df['Close'].pct_change() * 1000
tmp = df['High'].rolling(window).max()
df['Feature 4'] = (tmp-df['Open']) / (tmp-df['Low'].rolling(window).min()) * 100
df

,Market,Open,High,Low,Close,Volume,Value,No. Trades,Taker Buy Volume,Taker Buy Value,Feature 1,Feature 2,Feature 3,Feature 4
2018-01-01 00:00:00,BTCUSDT,13715.65,13715.65,13681.0,13707.92,2.844266,38931.004413,32,2.002554,27414.354115,70.406706,NaN,NaN,NaN
2018-01-01 00:01:00,BTCUSDT,13707.91,13707.91,13666.11,13694.92,2.113138,28910.982501,29,0.264134,3613.88418,12.49961,-0.257053,-0.948357,NaN
2018-01-01 00:02:00,BTCUSDT,13682.0,13694.94,13680.0,13680.0,1.346411,18431.84269,46,0.753332,10313.331785,55.951117,-0.362838,-1.089455,NaN
2018-01-01 00:03:00,BTCUSDT,13679.98,13679.98,13601.0,13645.99,11.589417,157962.177508,137,5.507146,75118.445437,47.518749,7.607637,-2.486111,NaN
2018-01-01 00:04:00,BTCUSDT,13645.98,13646.0,13576.28,13600.0,15.724566,213978.120703,191,9.739792,132561.881525,61.939973,0.356804,-3.370221,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-04-10 23:55:00,BTCUSDT,6827.0,6837.0,6827.0,6837.0,7.29528,49844.093534,141,5.167686,35312.022948,70.83602,-0.532374,1.464772,18.058064
2018-04-10 23:56:00,BTCUSDT,6835.02,6839.0,6833.61,6839.0,5.410635,36996.62508,106,4.080443,27901.723566,75.415233,-0.258338,0.292526,7.114286
2018-04-10 23:57:00,BTCUSDT,6837.35,6842.0,6835.01,6835.02,20.635987,141131.726927,110,16.446863,112486.809612,79.69991,2.813968,-0.581956,6.458333
2018-04-10 23:58:00,BTCUSDT,6842.0,6844.9,6835.03,6844.2,5.883241,40253.990341,87,2.605464,17831.988083,44.286202,-0.714904,1.343083,3.871829


In [37]:
x = df.dropna()[[f'Feature {i}' for i in range(1, 5)]]
x

,Feature 1,Feature 2,Feature 3,Feature 4
2018-01-01 00:59:00,76.095922,-0.504054,4.260812,61.041059
2018-01-01 01:00:00,61.400943,-0.809107,1.098380,58.109776
2018-01-01 01:01:00,9.540385,1.592045,-0.951722,42.362595
2018-01-01 01:02:00,42.997339,0.092650,0.000000,58.062298
2018-01-01 01:03:00,61.821167,0.205052,-0.745696,54.971464
...,...,...,...,...
2018-04-10 23:55:00,70.83602,-0.532374,1.464772,18.058064
2018-04-10 23:56:00,75.415233,-0.258338,0.292526,7.114286
2018-04-10 23:57:00,79.69991,2.813968,-0.581956,6.458333
2018-04-10 23:58:00,44.286202,-0.714904,1.343083,3.871829


In [39]:
t_train_start = datetime(2018, 1, 1, 1)
t_train_duration = timedelta(hours=6)
x_train = x.loc[t_train_start:t_train_start+t_train_duration-timedelta(minutes=1)]
A = 

,Feature 1,Feature 2,Feature 3,Feature 4
2018-01-01 01:00:00,61.400943,-0.809107,1.098380,58.109776
2018-01-01 01:01:00,9.540385,1.592045,-0.951722,42.362595
2018-01-01 01:02:00,42.997339,0.092650,0.000000,58.062298
2018-01-01 01:03:00,61.821167,0.205052,-0.745696,54.971464
2018-01-01 01:04:00,69.96112,0.878513,-0.742555,58.440487
...,...,...,...,...
2018-01-01 06:55:00,25.549727,-0.110769,-0.572097,8.619377
2018-01-01 06:56:00,41.886676,-0.323589,-0.138021,11.415225
2018-01-01 06:57:00,47.810175,0.915613,1.155178,7.427586
2018-01-01 06:58:00,53.652098,-0.669015,1.449201,3.451724


In [33]:
# df.dropna(inplace=True)
# df

,Market,Open,High,Low,Close,Volume,Value,No. Trades,Taker Buy Volume,Taker Buy Value,Feature 1,Feature 2,Feature 3,Feature 4
2018-01-01 00:59:00,BTCUSDT,13522.98,13574.05,13471.62,13529.01,8.045008,108840.254007,110,6.121923,82836.820623,76.095922,-0.504054,4.260812,61.041059
2018-01-01 01:00:00,BTCUSDT,13528.99,13571.74,13527.9,13543.87,1.535737,20808.440065,48,0.942957,12780.025236,61.400943,-0.809107,1.098380,58.109776
2018-01-01 01:01:00,BTCUSDT,13570.0,13570.0,13521.99,13530.98,3.980699,53935.697003,76,0.379774,5145.352856,9.540385,1.592045,-0.951722,42.362595
2018-01-01 01:02:00,BTCUSDT,13521.99,13530.98,13481.82,13530.98,4.349511,58824.596544,65,1.870174,25303.113683,42.997339,0.092650,0.000000,58.062298
2018-01-01 01:03:00,BTCUSDT,13530.98,13530.98,13486.53,13520.89,5.241386,70777.902377,58,3.240286,43765.342085,61.821167,0.205052,-0.745696,54.971464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-04-10 23:55:00,BTCUSDT,6827.0,6837.0,6827.0,6837.0,7.29528,49844.093534,141,5.167686,35312.022948,70.83602,-0.532374,1.464772,18.058064
2018-04-10 23:56:00,BTCUSDT,6835.02,6839.0,6833.61,6839.0,5.410635,36996.62508,106,4.080443,27901.723566,75.415233,-0.258338,0.292526,7.114286
2018-04-10 23:57:00,BTCUSDT,6837.35,6842.0,6835.01,6835.02,20.635987,141131.726927,110,16.446863,112486.809612,79.69991,2.813968,-0.581956,6.458333
2018-04-10 23:58:00,BTCUSDT,6842.0,6844.9,6835.03,6844.2,5.883241,40253.990341,87,2.605464,17831.988083,44.286202,-0.714904,1.343083,3.871829


array([[76.09592184276143, -0.504053747821106, 4.2608121820628675,
        61.04105943479927],
       [61.40094299600772, -0.8091068399186179, 1.0983804432105337,
        58.10977590126674],
       [9.54038474123413, 1.5920447316174582, -0.951722070575145,
        42.362594513952594],
       ...,
       [79.69990967681993, 2.81396767662206, -0.5819564263780386,
        6.458333333332828],
       [44.28620211131439, -0.7149038231125073, 1.3430831219221062,
        3.8718291054734983],
       [63.772158206064546, 1.120929433283457, -0.043832734285986774,
        3.9733333333327514]], dtype=object)